In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from openpyxl import load_workbook

In [2]:
def invokebrowser(url):
    '''
    instantiates chrome webdriver and browse given url and return webdriver 
    '''
    driver=webdriver.Chrome(r'C:\Users\divakar.kareddy\Hackathon\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    time.sleep(1)
    driver.maximize_window()
    return driver

In [9]:
def closedriver(driver):
    driver.close()

In [3]:
#Get the page data into soup object
def soup_website(driver):
    '''
    input-->webdriver instance
    output---> soup of the website invoked by driver'''
    soup=BeautifulSoup(driver.page_source)
    return soup

In [4]:
#Get the options available for player in the dropdown
def get_options(soup):
    '''
    input--->soup object of page
    output--->return the options present in the dropdown'''
    options=soup.find('select',{'class':'si-selectBox'}).find_all('option')
    columns=[options[i].text for i in range(len(options))]
    return columns

In [13]:
def scrapper(columns,excel):
    '''
    inputs
    columns--->list of options present in the dropdown
    #table_columns---> list of class of individual column of the table
    output
    created a excel sheet at pwd with Top10Players as name.
    
    '''
    table_colums=['sipk-lb-detailBlock sipk-lb-rank','sipk-lb-playerName','sipk-lb-detailBlock sipk-lb-team',\
                  'sipk-lb-detailBlock sipk-lb-matchedPlayed','sipk-lb-detailBlock sipk-lb-raidPoints']
    for option in range(1,len(columns)+1):
        driver.find_element_by_xpath('//*[@id="si_dropdown"]/option['+str(option)+']').click()
        time.sleep(1)
        page=BeautifulSoup(driver.page_source)
        table=page.find('div',{'class':'si-leadBoard-detail-wrap si-stats-partial-data'})
        headers_soup=table.find_all('div',{'class':'si-fullName'})
        headings= [headers_soup[i].find('span').text for i in range(len(headers_soup))]
        dis={}
        #to fetch columns data
        for col in range(len(table_colums)):
            #fetching top 10
            data=[]
            if col==2:#To fecth team name
                data=[table.find_all('a',{'class':table_colums[col]})[row].attrs['href'].replace('/teams/','')\
                   [:-10].replace('-',' ').rstrip(' ').title() for row in range(10)]
            else:
                 data=[table.find_all('div',{'class':table_colums[col]})[row].find_all('span')[0].text.strip()\
                                    .replace('.','')for row in range(10)]
            dis[headings[col]]=data
        page_df=pd.DataFrame(dis)
        if option==1:
            page_df.to_excel(excel,sheet_name=columns[option-1],engine='xlsxwriter',index=False)
        else:
            book = load_workbook(excel)
            writer = pd.ExcelWriter(excel, engine = 'openpyxl')
            writer.book = book
            page_df.to_excel(writer, sheet_name = columns[option-1])
            writer.save()
            writer.close()
    del page_df

In [11]:
url='https://www.prokabaddi.com'

In [12]:
driver=invokebrowser(url)
#click on stats link on prokabaddi website
driver.find_element_by_link_text('STATS').click()
#delay of 2 seconds
time.sleep(2)
#click on VIEW FULL TABLE
driver.find_element_by_link_text('VIEW FULL TABLE').click()
time.sleep(2)
scrapper(get_options(soup_website(driver)),'Top10Player.xlsx')
driver.find_element_by_id('team_Btn').click()
time.sleep(1)
scrapper(get_options(soup_website(driver)),'TopTeams.xlsx')
closedriver(driver)